# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [6]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/ham', 'ham'))


/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


Let's have a look at that DataFrame:

In [9]:
data.head()

,class,message
/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam/00077.c85b7442247d61308f15d86aa125ec28,spam,"OWOLABI & ASSOCIATE,\n\nFALOMO, IKOYI.\n\nLAGO..."
/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam/00352.19a8ba03f566612e0b9e124609d9dbd0,spam,This is a multi-part message in MIME format.\n...
/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam/00041.f1b3402799046db3c1f143a911dc085d,spam,Dear Sir or Madam:\n\n\n\nPlease reply to \n\n...
/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam/00343.37d895b3a54847548875136ad6b0192d,spam,Are you tired of spending a fortune on printer...
/home/nuttakan/Documents/Readable/DataScience/DataScience-Python3/emails/spam/00415.6faccf48ec514344fc850e8b3c154528,spam,\n\n\n\nEarn Extra Income From Home. Here is t...


<class 'pandas.core.frame.DataFrame'>


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [8]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [4]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.